In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import nltk

In [3]:
tweets = pd.read_csv("../datasets/tweets.csv", sep=",")

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/tweets.csv'

In [ ]:
tweets.head()

In [ ]:
len(tweets)

In [ ]:
testsätze = [
    "The sun is shining",
    "The weather is sweet",
    "The sun is shining and the weather is sweet"
]

In [ ]:
count = CountVectorizer()
docs = np.array(testsätze)
bag = count.fit_transform(docs)

### Vocabulary

In [ ]:
print(count.vocabulary_) ## Eigentlich müssten gewisse Preprocessing-Schritte vorgelagert sein --> Stop Words entfernen, UserNames etc.

### Merkmalsvektoren

One-Hot Codierte Arrays, welche anzeigen wie häufig bestimmte Wörter in einem Satz/Post/Dokument vorkommen

#### Monogramme

In [ ]:
print(count.vocabulary_)

In [ ]:
print(bag.toarray()) ##Merkmalsvektoren für jeden Satz

##### Bigramme

Durch N-Gramme oder diesem Fall Bigramme ist es möglich dem Text deutlich mehr Informationen zu entnehmen, da Wort- und Satzbau mit in Bezug genommen wird. Das Modell wird dadurch natürlich komplexer.

In [ ]:
count = CountVectorizer(ngram_range=(2,2))
docs = np.array(testsätze)
bag = count.fit_transform(docs)

In [ ]:
print(count.vocabulary_)

In [ ]:
print(bag.toarray())

### Beurteilung von Wortrelevanz

#### Tf-idf-Maß

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf = TfidfTransformer()
np.set_printoptions(precision=2) # 2 Nachkommastellen
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

### Tokenizer

Überführung von Wörtern in einzelne Wortabschnitte. Der Tokenizer von Porter überführt Wörter in ihre ursprüngliche Form um Vokabular zu verkleinern. Entweder entfernen von pre-/suffixen oder verwenden des Worstamms so wie es im Lexikon steht (Lemmatisierung).

In [ ]:
satz = "I want to go to the mall !"

satz.split(" ")

##### Stemming/Lemmatisierung

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run') ##Manchmal kommen nicht vorhandene Wörter als Ergebnis heraus --> Beispiel thus

Leider gibt es bei der nltk library keine deutsche Implementierung, das heißt es muss auf andere Tools/Libraries zurückgegriffen werden. Ein Stemming-Ansatz für deutsches Vokabular wird in folgendem beschrieben: https://textmining.wp.hs-hannover.de/Preprocessing.html

#### Stopwords

In [ ]:
nltk.download("stopwords")

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

new_list = []
words = tokenizer_porter('a runner likes running and runs a lot')
for w in words:
    if w not in stop:
        new_list.append(w)
new_list

##oder:
[w for w in tokenizer_porter('a runner likes running and runs a lot') if w not in stop]

### Machine Learning Pipeline

manueller Train-/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train,y_test = train_test_split(tweets.tweets,tweets.label,test_size=0.2, shuffle=True)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

params= {
    "logisticregression__penalty": ['l1', 'l2'], 
    "logisticregression__C": [1.0,10.0,100.0] 
}

pipeline = make_pipeline(
    TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,ngram_range=(1,1),stop_words=stop,tokenizer=tokenizer_porter),
    LogisticRegression(random_state=0)
)

In [ ]:
search = GridSearchCV(pipeline,params,cv=5)

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.score(X_test,y_test)

In [ ]:
search.predict(["This Machine Learning course was very good"])

#### Fazit

Das Modell ist ganz gut geeignet für dieses spezielle Datenset, jedoch fehlen dem Modell das Grundverständnis für beispielsweise Verneinungen, Sarkasmus, Satzbau etc. Dies könnte mit erweiterten Methoden angepasst werden. 